In [1]:
import pandas as pd
import numpy as np
import datetime
import time

pd.options.display.max_columns = None

In [2]:
df = pd.read_excel("2021_Accidentalidad.xlsx")
df.head(3)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14.0,Sin asistencia sanitaria,444976.463,4470508.259,N,NaN
1,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,444976.463,4470508.259,N,NaN
2,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,NaN,NaN,444976.463,4470508.259,N,NaN


# Duplicados

Eliminamos duplicados

In [3]:
df.drop_duplicates(inplace = True)

In [4]:
df.to_csv("df_limpio.csv")

# Tipado de columnas

In [5]:
df.dtypes

num_expediente                  object
fecha                   datetime64[ns]
hora                            object
localizacion                    object
numero                          object
cod_distrito                   float64
distrito                        object
tipo_accidente                  object
estado_meteorológico            object
tipo_vehiculo                   object
tipo_persona                    object
rango_edad                      object
sexo                            object
cod_lesividad                  float64
lesividad                       object
coordenada_x_utm               float64
coordenada_y_utm               float64
positiva_alcohol                object
positiva_droga                 float64
dtype: object

Cambiamos el tipado de columnas que queremos

In [6]:
df['numero'] = pd.to_numeric(df["numero"], errors='coerce').astype('Int64')
df['cod_distrito'] = pd.to_numeric(df["cod_distrito"], errors='coerce').astype('Int64')
df['cod_lesividad'] = pd.to_numeric(df["cod_lesividad"], errors='coerce').astype('Int64')
df["positiva_droga"] = df["positiva_droga"].astype("category")

In [7]:
df.shape

(39966, 19)

# Añadimos columnas

Añadimos columna de interés como el día de la semana

In [8]:
df['dia_semana'] = df['fecha'].dt.day_name()

In [9]:
df.head(2)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,dia_semana
0,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,Sin asistencia sanitaria,444976.463,4470508.259,N,NaN,Friday
1,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7,Asistencia sanitaria sólo en el lugar del acci...,444976.463,4470508.259,N,NaN,Friday


# Eliminamos columnas

Eliminamos la columna de "positiva_droga" por tener la mayoría de campos en nulo y no aportar gran valor

In [10]:
df_limpio = df.drop(["positiva_droga"], axis = 1)

In [11]:
df_limpio.head(2)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
0,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,14,Sin asistencia sanitaria,444976.463,4470508.259,N,Friday
1,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,7,Asistencia sanitaria sólo en el lugar del acci...,444976.463,4470508.259,N,Friday


# Gestión de Nulos

Eliminamos o sustituimos Nulos

In [12]:
df_limpio["positiva_alcohol"].unique()

array(['N', 'S', nan], dtype=object)

In [13]:
df_limpio["positiva_alcohol"].value_counts()

N    38523
S     1277
Name: positiva_alcohol, dtype: int64

In [14]:
df_limpio['positiva_alcohol'] = df['positiva_alcohol'].fillna("Sin especificar")

In [15]:
df_limpio.isnull().values.any()

True

In [16]:
df_limpio[df_limpio["distrito"] == "null" ]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana


In [34]:
nan_rows = df_limpio[df_limpio.isnull().any(1)]
nan_rows.head(2)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,dia_semana
2,2020S019534,2021-01-01,04:30:00,AVDA. PABLO NERUDA / CALL. LEONESES,57,13,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,Desconocido,Desconocido,<NA>,NaN,444976.463,4470508.259,N,Friday
16,2021S000006,2021-01-01,01:35:00,"CALL. JULIAN CAMARILLO, 21",21,20,SAN BLAS-CANILLEJAS,Otro,Nublado,Turismo,Conductor,De 45 a 49 años,Hombre,<NA>,NaN,446546.575,4476147.465,N,Friday


In [19]:
df_limpio.isnull().sum()

num_expediente              0
fecha                       0
hora                        0
localizacion                0
numero                   5623
cod_distrito                3
distrito                    3
tipo_accidente              4
estado_meteorológico     4156
tipo_vehiculo              54
tipo_persona                3
rango_edad                  0
sexo                        0
cod_lesividad           17489
lesividad               17489
coordenada_x_utm           12
coordenada_y_utm           12
positiva_alcohol            0
dia_semana                  0
dtype: int64

In [20]:
#df ['hora'] = pd. to_datetime.time (df['hora'])

In [21]:
df_pie = pd.DataFrame(df_limpio.groupby(['num_expediente', 'distrito'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())

df_pie.head(1)

/var/folders/hb/9zbycmb97bx0n1b4ggqr8s_c0000gn/T/ipykernel_26094/4186470162.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_pie = pd.DataFrame(df_limpio.groupby(['num_expediente', 'distrito'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())


,,tipo_vehiculo,tipo_accidente,tipo_persona
num_expediente,distrito,,,
2020S019534,PUENTE DE VALLECAS,5,5,5


In [36]:
df_pie.reset_index(inplace = True)
df_pie.head(2)

,index,num_expediente,distrito,tipo_vehiculo,tipo_accidente,tipo_persona
0,0,2020S019534,PUENTE DE VALLECAS,5,5,5
1,1,2021S000001,USERA,5,5,5


In [23]:
df_pie.shape

(17951, 5)

In [35]:
df_pie_1 = pd.DataFrame(df_pie.groupby(['distrito'])["num_expediente"].count())

df_pie_1.head(2)

,num_expediente
distrito,
ARGANZUELA,843
BARAJAS,315


In [25]:
#df_pie2 = pd.DataFrame(df_pie1.groupby(["distrito", "num_expediente"])["num_expediente"].count())

#df_pie2.head(20)